In [160]:
!pip freeze

absl-py==1.1.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.4.0
audioread==2.1.9
autograd==1.4
Babel==2.10.2
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.0
blis==0.7.7
bokeh==2.3.3
branca==0.5.0
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==4.2.4
catalogue==2.0.7
certifi==2022.6.15
cffi==1.15.0
cftime==1.6.0
chardet==3.0.4
charset-normalizer==2.0.12
click==7.1.2
cloudpickle==1.3.0
cmake==3.22.5
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.30
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill==0.3.5.1
distributed==1.25.3
dlib==19

In [161]:
!pip install pyspark unidecode airflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached airflow-0.6.tar.gz (1.2 kB)
ERROR: Could not find a version that satisfies the requirement airflow (from versions: 0.6)
ERROR: No matching distribution found for airflow


In [162]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
import pandas as pd
import requests
import json
import re
import unidecode
import sys


In [163]:
spark = SparkSession.builder.getOrCreate()

In [164]:
df_csv = spark.read.csv('/content/drive/MyDrive/engenharia_de_dados/pyspark/teste inter/AIR_CIA/*.csv',sep=';',
                         inferSchema=True, header=True)

df_csv.show()
#inferSchema=False é ideal pra camada rawzone

+--------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+
|        Razão Social|ICAO IATA|              CNPJ|   Atividades Aéreas|       Endereço Sede|            Telefone|              E-Mail| Decisão Operacional|Data Decisão Operacional|Validade Operacional|
+--------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+
|ABSA - AEROLINHAS...|   LTG M3|00.074.635/0001-33|TRANSPORTE AÉREO ...|AEROPORTO INTERNA...|      (11) 5582-8055|      gar@tam.com.br|       DECISÃO Nº 41|              22/04/2015|          23/04/2025|
|AEROSUL TÁXI AÉRE...|   ASO 2S|27.315.694/0001-02|SERVIÇOS AÉREOS P...|RODOVIA PR 218, K...|      (43) 3176-4030|operacoes@aerosul...|      DECISÃO Nº 282|              10/02/2021|       

In [165]:
def snake_case(df):
  new_cols=(unidecode.unidecode(column.replace(' ', '_').lower()) for column in df.columns)
  df_csv2 = df.toDF(*new_cols)
  return df_csv2

df_csv2 =  snake_case(df_csv)
df_companhia = df_csv2.withColumn('icao_empresa', split(df_csv2['icao_iata'], ' ').getItem(0)) \
       .withColumn('iata', split(df_csv2['icao_iata'], ' ').getItem(1))


In [166]:
df_json = spark.read.json('/content/drive/MyDrive/engenharia_de_dados/pyspark/teste inter/VRA/*.json')


In [167]:
def snake_case2(df):
    
  new_cols_json=(re.sub(r'(?<!^)(?=[A-Z])', '_', column).lower() for column in df.columns)
  df_json2 = df.toDF(*new_cols_json)
  new_cols_json = (unidecode.unidecode(column.replace('i_c_a_o', 'icao').lower()) for column in df_json2.columns)
  df_json2 = df_json2.toDF(*new_cols_json)
  return df_json2
df_json2 = snake_case2(df_json)


In [168]:
list_aerodromos_origem = [item[0] for item in df_json2.select('icao_aerodromo_origem').distinct().collect()]
list_aerodromos_destino = [item[0] for item in df_json2.select('icao_aerodromo_destino').distinct().collect()]
list_intesection_aerodromos = set(list_aerodromos_origem).intersection(set(list_aerodromos_destino))
list_dif_aerodromos = set(list_aerodromos_origem).symmetric_difference(set(list_aerodromos_destino))
list_aerodromos = set(list_dif_aerodromos).union(list_intesection_aerodromos)


In [169]:
url = "https://airport-info.p.rapidapi.com/airport"

headers = {
	"X-RapidAPI-Key": "***",
	"X-RapidAPI-Host": "airport-info.p.rapidapi.com"
}

def faz_request(url, headers):

  dic = {"data":[]}
  for icao in list_aerodromos:

    try:
      querystring = {"icao":icao}
      response = requests.request("GET", url, headers=headers, params=querystring)

      if 'error' not in json.loads(response.text):
        dic['data'].append(json.loads(response.text))
      else:
        pass
    except:
      print(sys.exc_info())
      raise ValueError('Erro na requisição')
  return dic
dict = faz_request(url, headers)

In [170]:
schema = list(dic['data'][0].keys())
df_aerodromos = pd.DataFrame(dic['data']) # como são 300 registros, faz sentido usar pandas
df_aerodromos2 = spark.createDataFrame(df_aerodromos)

In [171]:
## Criando as Tmepviews
df_companhia.createOrReplaceTempView("companhia")
df_aerodromos2.createOrReplaceTempView("aerodromos")
df_json2.createOrReplaceTempView("vra")

query = """ WITH temp_rota as (SELECT *, CONCAT(icao_aerodromo_origem,'-',icao_aerodromo_destino) AS rota 
            FROM vra),
            temp_comp_area as (select v.rota, c.razao_social, c.icao_empresa, v.icao_aerodromo_origem, v.icao_aerodromo_destino FROM companhia as c LEFT JOIN temp_rota as v ON c.icao_empresa = v.icao_empresa_aerea),
            temp_comp_area2 as (select *, a.name as name_origem  
                                FROM temp_comp_area as tc 
                                LEFT JOIN aerodromos as a on tc.icao_aerodromo_origem = a.icao 
                                ),
            temp_comp_area3 as (select tc2.rota, tc2.razao_social, tc2.icao_empresa, tc2.icao_aerodromo_origem, tc2.icao_aerodromo_destino, tc2.icao, tc2.name_origem , tc2.state as estado_origem, a.name as name_destino, a.state as estado_destino
                                FROM temp_comp_area2 as tc2 
                                LEFT JOIN aerodromos as a on tc2.icao_aerodromo_destino = a.icao 
                                )
            SELECT * FROM temp_comp_area3
             """

In [172]:
df_trusted = spark.sql(query)
df_trusted.createOrReplaceTempView("trusted")

In [173]:
query2 = """ with questao1 as (SELECT razao_social,icao_empresa,name_origem,name_destino,icao_aerodromo_origem,icao_aerodromo_destino,estado_origem,estado_destino,rota, count(rota), rank () over(PARTITION BY razao_social order by count(rota) DESC) as rank 
             FROM trusted
             GROUP BY razao_social,icao_empresa,name_origem,name_destino,icao_aerodromo_origem,icao_aerodromo_destino,estado_origem,estado_destino,rota)
            select * from questao1
            where rank = 1
             """
spark.sql(query2).show(100)

+--------------------+------------+--------------------+--------------------+---------------------+----------------------+-----------------+--------------------+---------+-----------+----+
|        razao_social|icao_empresa|         name_origem|        name_destino|icao_aerodromo_origem|icao_aerodromo_destino|    estado_origem|      estado_destino|     rota|count(rota)|rank|
+--------------------+------------+--------------------+--------------------+---------------------+----------------------+-----------------+--------------------+---------+-----------+----+
|ABSA - AEROLINHAS...|         LTG|Mariscal Sucre In...|Miami Internation...|                 SEQM|                  KMIA|        Pichincha|             Florida|SEQM-KMIA|        377|   1|
|AEROSUL TÁXI AÉRE...|         ASO|Hercílio Luz Inte...|     Caçador Airport|                 SBFL|                  SBCD|   Santa Catarina|      Santa Catarina|SBFL-SBCD|         52|   1|
|ASTA LINHAS AÉREA...|         SUL|Marechal Rondon I...

In [174]:
# O que devo fazer na query:

query3 = """ WITH temp_rota as (SELECT icao_aerodromo_origem,icao_aerodromo_destino,icao_empresa_aerea,situacao_voo,partida_real,chegada_real, CONCAT(icao_aerodromo_origem,'-',icao_aerodromo_destino) AS rota 
                                FROM vra),
            temp_comp_area as (SELECT c.razao_social,c.icao_empresa, t.icao_aerodromo_origem,t.icao_aerodromo_destino,t.icao_empresa_aerea,t.situacao_voo,t.rota, t.partida_real,t.chegada_real 
                               FROM temp_rota as t 
                               LEFT JOIN companhia as c 
                               ON t.icao_empresa_aerea = c.icao_empresa),
            temp_aerodromo_origem as (SELECT a.name as name_aeroporto_origem, a.icao as icao_aeroporto_origem, t.razao_social,t.icao_empresa, t.icao_aerodromo_origem,t.icao_empresa_aerea,t.situacao_voo,t.rota,t.partida_real
                              FROM aerodromos as a 
                              LEFT JOIN temp_comp_area AS t
                              ON a.icao = t.icao_aerodromo_origem
                              
             ),
             temp_aerodromo_destino as (SELECT a.name as name_aeroporto_destino, a.icao as icao_aeroporto_destino, t.razao_social,t.icao_empresa, t.icao_aerodromo_destino,t.icao_empresa_aerea,t.situacao_voo,t.rota, t.chegada_real
                              FROM aerodromos as a 
                              LEFT JOIN temp_comp_area AS t
                              ON a.icao = t.icao_aerodromo_destino
                              
             ),

            temp_origem as (SELECT name_aeroporto_origem, icao_aerodromo_origem, razao_social,situacao_voo, count(distinct rota) as rotas_origem, count(partida_real) as n_partidas
            FROM temp_aerodromo_origem
            WHERE situacao_voo = 'REALIZADO'
            GROUP BY icao_aerodromo_origem,name_aeroporto_origem,razao_social,situacao_voo
            ORDER BY rotas_origem desc),

            temp_destino as (SELECT name_aeroporto_destino, icao_aeroporto_destino,razao_social,situacao_voo, count(distinct rota) as rotas_destino, count(chegada_real) as n_chegadas
            FROM temp_aerodromo_destino
            WHERE situacao_voo = 'REALIZADO'
            GROUP BY icao_aeroporto_destino, name_aeroporto_destino,razao_social,situacao_voo
            ORDER BY rotas_destino desc)

            select to.name_aeroporto_origem, td.name_aeroporto_destino, to.icao_aerodromo_origem,td.icao_aeroporto_destino, to.razao_social,to.rotas_origem,td.rotas_destino, to.n_partidas, td.n_chegadas
            from temp_origem as to 
            LEFT JOIN temp_destino as td
            ON to.icao_aerodromo_origem = td.icao_aeroporto_destino
             """
spark.sql(query3).show()

+---------------------+----------------------+---------------------+----------------------+--------------------+------------+-------------+----------+----------+
|name_aeroporto_origem|name_aeroporto_destino|icao_aerodromo_origem|icao_aeroporto_destino|        razao_social|rotas_origem|rotas_destino|n_partidas|n_chegadas|
+---------------------+----------------------+---------------------+----------------------+--------------------+------------+-------------+----------+----------+
| Brasília Internat...|  Brasília Internat...|                 SBBR|                  SBBR|TAM LINHAS AÉREAS...|          37|           37|     16109|     16109|
| Brasília Internat...|  Brasília Internat...|                 SBBR|                  SBBR|TAM LINHAS AÉREAS...|          37|            5|     16109|       970|
| Brasília Internat...|  Brasília Internat...|                 SBBR|                  SBBR|TAM LINHAS AÉREAS...|          37|            4|     16109|       310|
| Brasília Internat...|  Bra

In [175]:
ARGS = {
    "owner": "airflow",
    "depends_on_past": False,
    "start_date": days_ago(6) # dias atrás para iniciar
}

with DAG(
    dag_id="tst_proj_dag",
    default_args=ARGS,
    schedule_interval="0 2 * * *", #executa todos os dias as 2 da manha
    max_active_runs=1 #executa uma instância por vez
    ) as dag:
    
  executa_raw = "executa tarefas da camada raw e salva num bucket-raw"

  executa_trusted = "executa tarefas da camada trusted e salva num bucket-trusted"

  executa_refined = "executa tarefas da camada refined e salva num bucket-refined"

  executa_raw >> executa_trusted >> executa_refined

SyntaxError: ignored